In [4]:
import pdb, sys, codecs, pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras.utils.np_utils import to_categorical

# Add the parent directory to PYTHONPATH so that we can use utils.py
sys.path.append('..')
import utils

In [6]:
# def build_word_vector_matrix(vector_file, n_words=None):
#     '''Read a GloVe array from sys.argv[1] and return its vectors and labels as arrays'''
#     np_arrays = []
#     labels_array = []

#     with codecs.open(vector_file, 'r', 'utf-8') as f:
#         for i, line in enumerate(f):
#             sr = line.split()
#             labels_array.append(sr[0])
#             np_arrays.append(np.array([float(j) for j in sr[1:]]))
#             if n_words and i == n_words - 1:
#                 break
#         return np.array(np_arrays), labels_array
            
# parse data and create embedding matrix and labels array
embeddings, labels, id_to_word, word_to_id = utils.build_word_vector_matrix('../data/embeddings/2/vectors_d50.txt')

# # create dictionaries from labels array for quick lookups
# id_to_word = dict(zip(range(len(labels)), labels))
# word_to_id = dict((v,k) for k,v in id_to_word.items())

In [7]:
def move_to_glove(move):
    if move in word_to_id:
        return embeddings[word_to_id[move]]
    else :
        # return the "I've never seen that" vector
#         print('NOT FOUND')
        return np.full(len(embeddings[0]), 1, dtype=np.float32) 

def encode_moves(moves, length):   
    encoded = [move_to_glove(m) for m in moves]
    while len(encoded) < length:
        encoded.append(np.zeros(len(embeddings[0])))
    return encoded

def load_data(file, num_to_load=None, split_ratio=0.75):
    if '.pickle' in file:
        # Parsing games with pgnparser takes a while, so I've pickled some
        with open(file, 'rb') as f:
            games = pickle.load(f)
        print('loaded {} games'.format(len(games)))
        
        # what is the longest game?
        longest_game = games[np.argmax([len(g.moves) for g in games])]
        max_half_turns = len(longest_game.moves)
        print('longest game is {} half turns'.format(max_half_turns))
        
        X, y = [], []
        counter = 0
        for game in games:
            for i, move in enumerate(game.moves):
                if move in word_to_id:
                    # for each move, encode all of the moves before it
                    X.append(encode_moves(game.moves[0:i], max_half_turns))
                    y.append(move)
#                     pdb.set_trace()
            counter += 1
            if counter % 25 == 0:
                print('loaded game {}'.format(counter))
            if num_to_load and counter == num_to_load:
                break  
                
#         test_embeddings(X, games, num_to_load) 
            
        num_test = round(len(X) * split_ratio)
        X_train, y_train = X[0:num_test], y[0:num_test]
        X_test, y_test = X[num_test:], y[num_test:] 
        
        return np.asarray(X_train), np.asarray(y_train), np.asarray(X_test), np.asarray(y_test)
    else: 
        #parse w/ pgnparser
        pass
    
def from_categorical(l):
    return np.argmax(l)

In [8]:
def test_embeddings(X, games, num_to_test):
    print('{} games'.format(len(games)))
    move_count = 0
    for i, game in enumerate(games):
        move_count = move_count + len(game.moves)
        if len(game.moves) > 1:
            move = game.moves[-2]
            if move in word_to_id:
                print('move {}'.format(move_count))
                print(move)
                print(len(X[move_count - 1]), len(game.moves) - 2)
                vec = embeddings[word_to_id[move]]
                matching_move = X[move_count - 1][len(game.moves) - 2]
    #             print(vec)
    #             print(matching_move)
                assert (vec == matching_move).all
            else:
                print('{} not in vocab'.format(move))
    #         print('tested {}'.format(i))
            if i == num_to_test - 2:
                break

In [9]:
X_train, y_train_labels, X_test, y_test_labels = load_data('../data/some.pickle', 1000)

loaded 17723 games
longest game is 315 half turns
loaded game 25
loaded game 50
loaded game 75
loaded game 100
loaded game 125
loaded game 150
loaded game 175
loaded game 200
loaded game 225
loaded game 250
loaded game 275
loaded game 300
loaded game 325
loaded game 350
loaded game 375
loaded game 400
loaded game 425
loaded game 450
loaded game 475
loaded game 500
loaded game 525
loaded game 550
loaded game 575
loaded game 600
loaded game 625
loaded game 650
loaded game 675
loaded game 700
loaded game 725
loaded game 750
loaded game 775
loaded game 800
loaded game 825
loaded game 850
loaded game 875
loaded game 900
loaded game 925
loaded game 950
loaded game 975
loaded game 1000


In [6]:
X_train = np.resize(X_train, (X_train.shape[0], 3150))
X_test = np.resize(X_test, (X_test.shape[0], 3150))

## label one-hot encoding
# y_train = to_categorical([word_to_id[w] for w in y_train_labels], len(labels))
# y_test = to_categorical([word_to_id[w] for w in y_test_labels], len(labels))

## label int encoding
y_train = [word_to_id[w] for w in y_train_labels]
y_test = [word_to_id[w] for w in y_test_labels]

# print(X_train.shape, len(y_train))
# print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [7]:
clf = KNeighborsClassifier()
clf = SVC()
clf.fit(X_train[0:1000], y_train[0:1000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
predicted = clf.predict(X_test[0:100])

In [9]:
predicted = [id_to_word[index] for index in [from_categorical(c) for c in predicted]]

In [10]:
real = y_test_labels[0:100]

In [11]:
for r, p in dict(zip(real, predicted)).items():
    print(r, p)

Nd1 O-O
Nc3 O-O
Rfd8 O-O
Rxd8+ O-O
Rg5 O-O
Nf6 O-O
Ke7 O-O
1-0 O-O
Ree3 O-O
hxg4 O-O
Rgf2 O-O
Rd4 O-O
Re3 O-O
d4 O-O
Rd1 O-O
Rd2 O-O
h3 O-O
Re2 O-O
Kb3 O-O
Re1 O-O
c5 O-O
Bg6 O-O
Re8+ O-O
Rfd3 O-O
a5 O-O
Rb1 O-O
Kf8 O-O
Rf4 O-O
Bg7 O-O
Be5 O-O
Qxd4 O-O
Bd4 O-O
Be3 O-O
Rxd8 O-O
Rg1 O-O
Rxa2 O-O
g6 O-O
c6 O-O
Na6 O-O
Re5 O-O
Nf3 O-O
h5 O-O
f3 O-O
Rff3 O-O
c4 O-O
e5 O-O
Rff2 O-O
dxe5 O-O
cxb4 O-O
Na4 O-O
d6 O-O
Bxe5 O-O
e4 O-O
Qb6 O-O
exd4 O-O
Rxd1 O-O
Rd3 O-O
Qxd1 O-O
Ng4 O-O
bxc3 O-O
Nc5 O-O
a4 O-O
b5 O-O
Be2 O-O
Rge3 O-O
g5 O-O
Be6 O-O
a3 O-O
Rh1 O-O
b4 O-O
Bf4 O-O
Bf1 O-O
Rd8 O-O
Rb3 O-O
Rc1 O-O
Kf2 O-O
Nxd4 O-O
Rxc3 O-O
Rgg2 O-O
Rg2 O-O
Ke1 O-O
O-O O-O
Nxc3 O-O
